In [11]:
import hopsworks

project = hopsworks.login()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/14502


In [29]:
!python3 -m streamlit run streamlit_app.py


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://192.168.20.30:8501

  For better performance, install the Watchdog module:

  $ xcode-select --install
  $ pip install watchdog
            
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/14502

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/14502

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/14502
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/14502
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection grac

In [1]:
continent = "EU"

In [2]:
import datetime
import time
import requests
import pandas as pd
import json
import hopsworks

from functions import *

import warnings
warnings.filterwarnings("ignore")

# from dotenv import load_dotenv
# load_dotenv()


with open('target_cities.json') as json_file:
    target_cities = json.load(json_file)


def get_batch_data_from_fs(td_version, date_threshold):
    print(f"Retrieving the Batch data since {date_threshold}")
    feature_view.init_batch_scoring(training_dataset_version=td_version)

    batch_data = feature_view.get_batch_data(start_time=date_threshold)
    return batch_data


def parse_aq_data(last_dates_dict, today):
    start_of_cell = time.time()
    df_aq_raw = pd.DataFrame()
    
    print("Parsing started...")
    # for continent in target_cities:
    for city_name, coords in target_cities[continent].items():
        df_ = get_aqi_data_from_open_meteo(city_name=city_name,
                                           coordinates=coords,
                                           start_date=last_dates_dict[city_name],
                                           end_date=str(today))
        df_aq_raw = pd.concat([df_aq_raw, df_]).reset_index(drop=True)
    end_of_cell = time.time()
    print("-" * 64)
    print(f"Parsed new PM2.5 data for ALL locations up to {str(today)}.")
    print(f"Took {round(end_of_cell - start_of_cell, 2)} sec.\n")
    return df_aq_raw


def parse_weather(last_dates_dict, today):
    df_weather_update = pd.DataFrame()
    start_of_cell = time.time()
    
    print("Parsing started...")
    # for continent in target_cities:
    for city_name, coords in target_cities[continent].items():
        df_ = get_weather_data_from_open_meteo(city_name=city_name,
                                               coordinates=coords,
                                               start_date=last_dates_dict[city_name],
                                               end_date=str(today),
                                               forecast=True)
        df_weather_update = pd.concat([df_weather_update, df_]).reset_index(drop=True)

    end_of_cell = time.time()
    print(f"Parsed new weather data for ALL cities up to {str(today)}.")
    print(f"Took {round(end_of_cell - start_of_cell, 2)} sec.\n")
    return df_weather_update



if __name__=="__main__":
    project = hopsworks.login()
    fs = project.get_feature_store()
    print("✅ Logged in successfully!")

    feature_view = fs.get_feature_view(
        name='air_quality_fv',
        version=1
    )

    # I am going to load data for of last 60 days (for feature engineering)
    today = datetime.date.today()
    date_threshold = today - datetime.timedelta(days=60)
    
    print("Getting the batch data...")
    # batch_data = get_batch_data_from_fs(td_version=1,
    #                                     date_threshold=date_threshold)
    
    batch_data = pd.read_csv("batch_data.csv")

    print("Retreived batch data.")


    last_dates_dict = batch_data[["date", "city_name"]].groupby("city_name").max()
    last_dates_dict.date = last_dates_dict.date.astype(str)
    # here is a dictionary with city names as keys and last updated date as values
    last_dates_dict = last_dates_dict.to_dict()["date"]  
    
    df_aq_raw = parse_aq_data(last_dates_dict, today)

    # we need the previous data to calculate aggregation functions
    df_aq_update = pd.concat([
        batch_data[df_aq_raw.columns],
        df_aq_raw
    ]).reset_index(drop=True)
    df_aq_update = df_aq_update.drop_duplicates(subset=['city_name', 'date'])

    print(df_aq_update.tail(7))

    print('\n🛠 Feature Engineering the PM2.5')

    ###
    df_aq_update['date'] = pd.to_datetime(df_aq_update['date'])
    df_aq_update = feature_engineer_aq(df_aq_update)
    df_aq_update = df_aq_update.dropna()
    
    print(df_aq_update.groupby("city_name").max().tail(7))
    print("✅ Success!")
    ###

    print(3 * "-")
    print('\n🌤📆  Parsing Weather data')

    df_weather_update = parse_weather(last_dates_dict, today)
    print(df_weather_update.groupby("city_name").max().tail(7))
    print("✅ Successfully parsed!")

    df_aq_update.date = df_aq_update.date.astype(str)
    df_weather_update.date = df_weather_update.date.astype(str)
    
    print("Connecting to feature groups...")
    air_quality_fg = fs.get_or_create_feature_group(
        name = 'air_quality',
        version = 1
    )
    weather_fg = fs.get_or_create_feature_group(
        name = 'weather',
        version = 1
    )

    df_aq_update.date = pd.to_datetime(df_aq_update.date)
    df_weather_update.date = pd.to_datetime(df_weather_update.date)

    df_aq_update["unix_time"] = df_aq_update["date"].apply(convert_date_to_unix)
    df_weather_update["unix_time"] = df_weather_update["date"].apply(convert_date_to_unix)

    df_aq_update.date = df_aq_update.date.astype(str)
    df_weather_update.date = df_weather_update.date.astype(str)

    air_quality_fg.insert(df_aq_update,
                          write_options={'wait_for_job': False})
    print("Created job to insert parsed PM2.5 data into FS...")
    print("Inserting into air_quality fg.")

    weather_fg.insert(df_weather_update,
                      write_options={'wait_for_job': False})
    print("Created job to insert parsed weather data into FS...")
    print("Inserting into weather fg.")

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/14502
Connected. Call `.close()` to terminate connection gracefully.
✅ Logged in successfully!
Getting the batch data...
Retreived batch data.
Parsing started...
Processed PM2_5 for Amsterdam since 2023-04-21 till 2023-04-21.
Took 0.14 sec.

Processed PM2_5 for Athina since 2023-04-21 till 2023-04-21.
Took 0.13 sec.

Processed PM2_5 for Berlin since 2023-04-21 till 2023-04-21.
Took 0.14 sec.

Processed PM2_5 for Gdansk since 2023-04-21 till 2023-04-21.
Took 0.14 sec.

Processed PM2_5 for Kraków since 2023-04-21 till 2023-04-21.
Took 0.14 sec.

Processed PM2_5 for London since 2023-04-21 till 2023-04-21.
Took 0.13 sec.

Processed PM2_5 for Madrid since 2023-04-21 till 2023-04-21.
Took 0.14 sec.

Processed PM2_5 for Marseille since 2023-04-21 till 2023-04-21.
Took 0.13 sec.

Processed PM2_5 for Milano since 2023-04-21 till 2023-04-21.
Took 0.14 sec.

Proc

FeatureStoreException: Features are not compatible with Feature Group schema: 
 - pm_2_5_previous_1_day (type: 'double') is missing from input dataframe.
 - pm_2_5_previous_2_day (type: 'double') is missing from input dataframe.
 - pm_2_5_previous_3_day (type: 'double') is missing from input dataframe.
 - pm_2_5_previous_4_day (type: 'double') is missing from input dataframe.
 - pm_2_5_previous_5_day (type: 'double') is missing from input dataframe.
 - pm_2_5_previous_6_day (type: 'double') is missing from input dataframe.
 - pm_2_5_previous_7_day (type: 'double') is missing from input dataframe.
 - mean_7_days (type: 'double') is missing from input dataframe.
 - mean_14_days (type: 'double') is missing from input dataframe.
 - mean_28_days (type: 'double') is missing from input dataframe.
 - std_7_days (type: 'double') is missing from input dataframe.
 - exp_mean_7_days (type: 'double') is missing from input dataframe.
 - exp_std_7_days (type: 'double') is missing from input dataframe